In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)
from matplotlib.pyplot import cm
import random
from scipy.stats import ttest_ind

In [2]:
for i in range(1, 11):
    
    #generate a G(n,m) type random graph with n = 1460 and m = 974
    
    g = nx.gnm_random_graph(1460, 974)
    
    # save each graph in edge list format 
    
    with open('random_graph_' + str(i) + '.txt', 'w') as fout:
        for e in g.edges():
            fout.write(str(e[0]) + ' ' + str(e[1]) + '\n')

In [ ]:
network1 = nx.read_edgelist('dataset.txt', nodetype = int)
cn = nx.read_edgelist('cn.txt', nodetype = int)

print('average degree of network:', round(len(network1.edges())/len(network1.nodes()), 3))
print('average degree cn:', round(len(cn.edges())/len(cn.nodes()), 3))

for i in range(1, 11):
    h = nx.read_edgelist('random_graph_' + str(i) + '.txt', nodetype = int)
    print('average degree RG'+str(i)+':', round(len(h.edges())/len(h.nodes()), 3))

In [ ]:
csch = nx.closeness_centrality(network1) 
[ys, bs] = np.histogram(list(csch.values()))
xs = [(bs[j] + bs[j+1])/2 for j in range(len(bs)-1)]


ccn = nx.closeness_centrality(cn) 
[yf, bf] = np.histogram(list(cfmh.values()))
xf = [(bf[j] + bf[j+1])/2 for j in range(len(bf)-1)]

plt.plot(xs, ys, 'o-', color = 'red')
plt.plot(xf, yf, 'o-', color = 'green')
plt.text(0.03, 400, '$cn$', style ='italic', color = 'green')
plt.text(0.03, 300, '$network1$', style ='italic', color = 'red')
plt.xlabel('$centrality$')
plt.ylabel("$frequency$")

In [ ]:
colours = cm.YlOrBr(np.linspace(0,1,10))

for i in range(1, 11):
    h = nx.read_edgelist('random_graph_' + str(i) + '.txt', nodetype = int)
    cc = nx.closeness_centrality(h) 
    [yh, bh] = np.histogram(list(cc.values()))
    xh = [(bh[j] + bh[j+1])/2 for j in range(len(bh)-1)]
    plt.plot(xh, yh, '.-', color = colours[i-1])
    
plt.plot(xs, ys, 'o-', color = 'red')
plt.text(0.03, 400, '$network1$', style ='italic', color = 'red')
plt.xlabel('$centrality$')
plt.ylabel("$frequency$")

In [ ]:
for i in range(1, 11):
    h = nx.read_edgelist('random_graph_' + str(i) + '.txt', nodetype = int)
    cc = nx.closeness_centrality(h) 
    [yh, bh] = np.histogram(list(cc.values()))
    xh = [(bh[j] + bh[j+1])/2 for j in range(len(bh)-1)]
    plt.plot(xh, yh, '.-', color = colours[i-1])
    
plt.plot(xs, ys, 'o-', color = 'red')
plt.plot(xf, yf, 'o-', color = 'green')
plt.text(0.03, 420, '$fmh$', style ='italic', color = 'green')
plt.text(0.03, 350, '$school$', style ='italic', color = 'red')
plt.xlabel('$ $centrality$')
plt.ylabel("$frequency$")

In [8]:
#Simulate F-R-E-S Model model on the networks

def epi_main(num, graph, inf_rate, rec_rate):


    fr_inf = []
    att_rate = []

    init = 1   #no. of individuals infected at time t = 0

    for m in range(0, num):
        n_infected = [1]
        n_susceptible = [graph.number_of_nodes()-1]
        new_infected = []
        n_removed = [0]

        state = [0] * (max(graph.nodes())+1)

        # to not run into errors if all the node ids are not continuous values
        state[0] = 5
        for i in range(len(state)):
            if i not in graph.nodes():
                state[i] = 5

        infected = random.sample((graph.nodes()), init)
        #    print(infected)

        for i in range(len(infected)):
            state[infected[i]] = 1

        while len(infected) > 0:

        ### Spreading infection

            for j in infected:
                neighbors = graph.neighbors(j)

                for k in neighbors:
                    if state[k] == 0 and random.random() < inf_rate:
                        state[k] = 1
                        new_infected.append(k)

            ### Recovering from infection
            for k in infected[:]:             #need to access new changed list every time, so slice assignment
                if random.random() < rec_rate:
                    state[k] = 2
                    infected.remove(k)



            infected = new_infected + infected
            new_infected = []
            n_infected.append(state.count(1))
            n_susceptible.append(state.count(0))
            n_removed.append(state.count(2))

        # gives the fraction infected
        fr_inf.append(list(n_infected[t]/graph.number_of_nodes() for t in range(len(n_infected)))) 
        
        # gives the attack rate
        att_rate.append(n_removed[-1]/graph.number_of_nodes()) 
        
    return att_rate, fr_inf

In [ ]:
num_sim = 10000

inf_rate = 0.8
rec_rate = 1

rng_att_rate = []

sch_att_rate, sch_inf = epi_main(num_sim, school, inf_rate, rec_rate)

plt.figure(figsize=(10, 8))

for i in range(1, 11):
    h = nx.read_edgelist('random_graph_' + str(i) + '.txt', nodetype = int)
    ar, rng_inf = epi_main(num_sim, h, inf_rate, rec_rate)
    rng_att_rate.append(ar)
    
    rng_inf_mean = [np.mean([x[i] for x in rng_inf if len(x) > i]) for i in range(len(max(rng_inf, key = len)))]
    strg = [np.std([x[i] for x in rng_inf if len(x) > i]) for i in range(len(max(rng_inf, key = len)))]

    plt.errorbar(range(len(rng_inf_mean)), rng_inf_mean, yerr = strg/np.sqrt(len(rng_inf_mean)), 
             color = colours[i-1], markersize = 5, ecolor = colours[i-1],
             fmt='-o', markeredgewidth = 2, capsize = 8, elinewidth=1)

av_atr_rng = [np.mean(rng_att_rate[i]) for i in range(len(rng_att_rate))]

#print('average attack rate for probability of transmission, 0.8 for random graphs = ', round(np.mean(av_atr_rng), 4))
#print('std. dev. sch', round(np.std(av_atr_rng)/np.sqrt(num_sim),4))    



sch_inf_mean = [np.mean([x[i] for x in sch_inf if len(x) > i]) for i in range(len(max(sch_inf, key = len)))]
sts = [np.std([x[i] for x in sch_inf if len(x) > i]) for i in range(len(max(sch_inf, key = len)))]

plt.errorbar(range(len(sch_inf_mean)), sch_inf_mean, yerr = sts/np.sqrt(len(sch_inf_mean)), 
             color = 'red', markersize = 5, ecolor = 'red',
             fmt='-o', markeredgewidth=2, capsize=10, elinewidth=2)

plt.text(70, 0.007, '$network1$', style ='italic', color = 'red', fontsize = 25)
plt.xlabel('$time$')
plt.ylabel('$fraction$  $of$ $infected$')

print('average attack rate for probability of transmission, 0.8 for network1 = ', round(np.mean(sch_att_rate),4))
print('std. dev. sch', round(np.std(sch_att_rate)/np.sqrt(num_sim),4))

In [ ]:
num_sim = 10000

inf_rate = 0.8
rec_rate = 1


sch_att_rate, sch_inf = epi_main(num_sim, school, inf_rate, rec_rate)
fmh_att_rate, fmh_inf = epi_main(num_sim, fmh, inf_rate, rec_rate)

print('average attack rate for probability of transmission, 0.8 for network1 = ', round(np.mean(sch_att_rate),4))
print('std. dev. sch', round(np.std(sch_att_rate)/np.sqrt(num_sim),4))

print('average attack rate for probability of transmission, 0.8 for cn = ', round(np.mean(fmh_att_rate),4))
print('std. dev. cn', round(np.std(fmh_att_rate)/np.sqrt(num_sim),4))